# EDGAR INDICES PROJECT

_**EDGAR Indices project :**_
This project entails writing Python code that efficiently processes EDGAR index files to construct a data frame, enabling users to perform basic searches and analysis.
Our Main Aim Was To :
1) Produce Python code that addresses a specific institutional need.
2) Learn about an important financial data repository and its contents as well as access.
3) Download index files (daily, quarterly) directly from the SEC's EDGAR site (https://www.sec.gov/os/accessing-edgar-data).
4) Analyse the available data using different parameters and try to draw conclusion.

**About EDGAR (*Electronic Data Gathering, Analysis and Retrival* ) :**

* The EDGAR system serves as a primary platform for companies and entities required to file information with the U.S.Securities and Exchange Commission(SEC).

* The system is designed to automatically collect, validate, index, accept, and forward submissions.

* Entities such as public companies, mutual funds, and other investment funds use EDGAR to submit regulatory filings, such as quarterly and annual reports (10-Q, 10-K), prospectuses, and much more.

* As part of its mandate, the SEC makes most of these filings available to the public through the Edgar database, which can be accessed online.

* For this project, we excluded the XBRL filings. Parse index files, and create a data frame that will have all the content from index files.

* Allow users query the dataframe based on Form type / CIK / Company Name to generate a list of filings for the filters the user entered. 

* The goal is to be able to get a certain filing or a list of filings. Display a filing with a given/chosen URL.

# Code to download all the files which are available on the EDGAR website.

In [1]:
import requests
import os
import time

# Base URL for the SEC EDGAR full index
base_url = 'https://www.sec.gov/Archives/edgar/full-index/'

# Function to download the file, now includes headers parameter
def download_file(url, path, headers):
    with requests.get(url, headers=headers, stream=True) as r:
        r.raise_for_status()
        with open(path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

# Prompt for start and end year
start_year = int(input("Enter the start year (YYYY): "))
end_year = int(input("Enter the end year (YYYY): "))
save_dir = input('Please Input Path to Your Directory to Download Files:')

# Add your user-agent string here
headers = {'User-Agent': 'useremail@email.com'}

# Iterate over each year and quarter within the specified range
for year in range(start_year, end_year + 1):
    for quarter in ['QTR1', 'QTR2', 'QTR3', 'QTR4']:
        file_url = f"{base_url}{year}/{quarter}/company.idx"
        save_path = os.path.join(save_dir, f"{year}_{quarter}_company.idx")

        print(f"Attempting to download {file_url}...")

        # Make the download attempt
        try:
            download_file(file_url, save_path, headers)
            print(f"Successfully downloaded {file_url}")
        except Exception as e:
            print(f"Failed to download {file_url}. Error: {e}")

        # Respect the SEC's rate limiting
        time.sleep(1)  # Sleep for 1 second to avoid hitting rate limit

print("All requested files have been attempted to download.")

Attempting to download https://www.sec.gov/Archives/edgar/full-index/2020/QTR1/company.idx...
Successfully downloaded https://www.sec.gov/Archives/edgar/full-index/2020/QTR1/company.idx
Attempting to download https://www.sec.gov/Archives/edgar/full-index/2020/QTR2/company.idx...
Successfully downloaded https://www.sec.gov/Archives/edgar/full-index/2020/QTR2/company.idx
Attempting to download https://www.sec.gov/Archives/edgar/full-index/2020/QTR3/company.idx...
Successfully downloaded https://www.sec.gov/Archives/edgar/full-index/2020/QTR3/company.idx
Attempting to download https://www.sec.gov/Archives/edgar/full-index/2020/QTR4/company.idx...
Successfully downloaded https://www.sec.gov/Archives/edgar/full-index/2020/QTR4/company.idx
Attempting to download https://www.sec.gov/Archives/edgar/full-index/2021/QTR1/company.idx...
Successfully downloaded https://www.sec.gov/Archives/edgar/full-index/2021/QTR1/company.idx
Attempting to download https://www.sec.gov/Archives/edgar/full-index/2

# Get the required files based on the User interest

Here we define functions to get the company data based on the User search type i.e.,

1) Time Period of Filings
2) Company Name
3) CIK

In [17]:
import pandas as pd  # Importing pandas for data manipulation
import os  # Importing os for file and directory operations

# Adjusting pandas display options for more optimized data viewing
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.width', None)  # Automatically adjust display width to terminal size
pd.set_option('display.max_colwidth', None)  # Display full content of each cell

# Function to load EDGAR index files from a specified year and directory
def load_data(year, source_dir):
    colspecs = [(0, 62), (62, 74), (74, 86), (86, 98), (98, None)]  # Column widths in fixed-width file
    column_names = ['Company Name', 'Form Type', 'CIK', 'Date Filed', 'Filename']  # Names of columns
    dataframe_collection = []  # List to store dataframes for each quarter

    # Loop through each quarter
    for quarter in ['QTR1', 'QTR2', 'QTR3', 'QTR4']:
        file_name = f'{year}_{quarter}_company.idx'  # Construct filename for each quarter
        file_path = os.path.join(source_dir, file_name)  # Create full path to the file

        # Check if the file exists before trying to open it
        if os.path.exists(file_path):
            try:
                # Read fixed-width file with specified columns and skip header rows using skiprows
                temp_df = pd.read_fwf(file_path, colspecs=colspecs, skiprows=9, names=column_names)
                dataframe_collection.append(temp_df)  # Add dataframe to the collection
            except UnicodeDecodeError as e:
                # Handle errors that occur if the file has encoding issues
                print(f'Error reading {file_name}: {e}')
                continue
            except Exception as e:
                # Handle other unexpected errors during file reading
                print(f'An unexpected error occurred while reading {file_name}: {e}')
                continue

    # Check if any data was loaded successfully
    if not dataframe_collection:
        print("No data was loaded. Please check your file paths and names.")
        return pd.DataFrame()  # Return an empty DataFrame if no data was loaded

    # Concatenate all quarterly DataFrames into one DataFrame
    all_data_df = pd.concat(dataframe_collection, ignore_index=True)
    all_data_df.columns = all_data_df.columns.str.strip()  # Strip any leading/trailing whitespace from column names
    return all_data_df  # Return the concatenated DataFrame

# Function to filter the DataFrame by company name or CIK
def find_companies_or_cik(all_data_df, search_term, search_type):
    try:
        if search_type == 'name':
            # Filter DataFrame by company name, ignoring case
            matching = all_data_df[all_data_df['Company Name'].str.contains(search_term, case=False, na=False)]
        else:  # search by 'cik'
            # Filter DataFrame by CIK
            matching = all_data_df[all_data_df['CIK'].astype(str) == str(search_term)]
        # Return filtered DataFrame with duplicate entries removed
        return matching[['Company Name', 'CIK']].drop_duplicates().reset_index(drop=True)
    except Exception as e:
        # Handle any errors that occur during filtering
        print(f"An error occurred during search: {e}")
        return pd.DataFrame()  # Return an empty DataFrame if an error occurs

# Function to get all filings for a specific CIK in a given year
def get_filings_for_company(all_data_df, cik, year):
    try:
        # Filter DataFrame for filings by the specified CIK and year
        filings_df = all_data_df[
            (all_data_df['CIK'] == cik) &
            (all_data_df['Date Filed'].str.contains(str(year)))
        ].reset_index(drop=True)
        return filings_df  # Return the filtered DataFrame
    except Exception as e:
        # Handle any errors during the filtering
        print(f"An error occurred while fetching filings for company CIK {cik}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame if an error occurs

# Function to display filings and allow the user to select one to view its URL
def display_and_choose_filings(filings_df):
    try:
        if filings_df.empty:
            # Check if the DataFrame is empty (no filings found)
            print("No filings found for this company.")
            return None

        # Display available filings for selection
        print(filings_df[['Form Type', 'Date Filed', 'Filename']])
        index_to_display = int(input("Enter the index number of the filing to display: "))
        # Validate user input
        if index_to_display >= len(filings_df):
            print("Invalid index number. Please enter a valid index from the list.")
            return None

        # Get the selected filing based on user input
        selected_filing = filings_df.iloc[index_to_display]
        # Construct the URL to view the filing
        filing_url = 'https://www.sec.gov/Archives/' + selected_filing['Filename']
        print(f"You can view the filing at the following URL:\n{filing_url}")
        return filing_url  # Return the URL
    except ValueError:
        # Handle invalid numeric input
        print("Invalid input. Please enter a numerical index.")
        return None
    except Exception as e:
        # Handle any other errors
        print(f"An error occurred while selecting filings: {e}")
        return None

if __name__ == "__main__":
    try:
        # Prompt user for the year of the filings to search
        year = int(input("Enter the exact year of the filings you want to search: "))
        print('Your Range of Data is:', year)
        
        # Prompt user for the type of search (name or CIK)
        search_type = input("Enter your choice of search : 'name' or 'CIK'? : ").lower()

        # Get the search term from the user based on the search type
        if search_type == 'name':
            search_term = input("Enter the company name to search for: ")
        else:
            search_term = input("Enter the CIK to search for: ").lstrip('0')

        # Ask user for the directory where the .idx files are stored
        source_dir = input('Please input the directory of your .idx files : ')
        
        # Load data from the specified year and directory
        user_data_df = load_data(year, source_dir)
        if user_data_df.empty:
            # Check if data was loaded successfully
            print("Data loading failed. Please check the file paths and year input.")
        else:
            print("Data loaded successfully. Here are the column names:")
            print(user_data_df.columns.tolist())

            # Validate the search type
            if search_type not in ['name', 'cik']:
                print("Invalid search type entered. Please restart and enter either 'name' or 'CIK'.")
            else:
                # Find matching companies or CIKs based on the search term and type
                matching_entities = find_companies_or_cik(user_data_df, search_term, search_type)
                if matching_entities.empty:
                    print(f"No matching entities found for the given {search_type}.")
                else:
                    print("Matching companies/CIKs:")
                    print(matching_entities)

                    # Handle filings retrieval based on the search type
                    if search_type == 'cik':
                        filings_df = get_filings_for_company(user_data_df, search_term, year)
                        display_and_choose_filings(filings_df)
                    elif search_type == 'name':
                        # Allow user to select a company from the list
                        company_index = int(input("Enter the index number of the company to display filings for: "))
                        if company_index not in matching_entities.index:
                            print("Invalid index number. Please enter a valid index from the list.")
                        else:
                            # Retrieve and display filings for the selected company
                            selected_cik = matching_entities.at[company_index, 'CIK']
                            filings_df = get_filings_for_company(user_data_df, selected_cik, year)
                            display_and_choose_filings(filings_df)

    except ValueError:
        # Handle errors caused by incorrect numeric input
        print("You have entered an invalid year or index number. Please restart and enter a valid number.")
    except Exception as e:
        # Handle any other exceptions that may occur
        print(f"An unexpected error occurred: {e}")

Your Range of Data is: 2022
Data loaded successfully. Here are the column names:
['Company Name', 'Form Type', 'CIK', 'Date Filed', 'Filename']
Matching companies/CIKs:
                                     Company Name      CIK
0                            APPLE LANE GROUP LLC  1268083
1                                  APPLE ROBERT F  1054942
2                               APPLEGATE DIANE L  1628199
3                             APPLETON GROUP, LLC  1454759
4                        APPLETON PARTNERS INC/MA  1055290
5                    Apple Hospitality REIT, Inc.  1418121
6                                      Apple Inc.   320193
7                                  Apple Robert E  1382697
8                                Appleton William  1431078
9                                   Appleyard Ian  1740298
10                                  KAPPLER ANN M  1280546
11                                Kapples John W.  1663408
12                   MAUI LAND & PINEAPPLE CO INC    63330
13  N

# Code to Create a Dataframe and Save it as *CSV* file in Local Repository

In [2]:
import pandas as pd
import os

# Adjusting pandas display options for more optimized data viewing
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.width', None)  # Automatically adjust display width to terminal size
pd.set_option('display.max_colwidth', None)  # Display full content of each cell

#Load data from all EDGAR index files in the specified director
def load_data_from_directory(source_dir):
    colspecs = [(0, 62), (62, 74), (74, 86), (86, 98), (98, None)]
    column_names = ['Company Name', 'Form Type', 'CIK', 'Date Filed', 'Filename']
    dataframe_collection = []

    # Iterate over each file in the directory
    for file_name in os.listdir(source_dir):
        if file_name.endswith('.idx'):  # Check for .idx files
            file_path = os.path.join(source_dir, file_name)
            try:
                # Read fixed-width file with specified columns and skip header rows
                temp_df = pd.read_fwf(file_path, colspecs=colspecs, skiprows=9, names=column_names)
                dataframe_collection.append(temp_df)
            except UnicodeDecodeError as e:
                print(f'Error reading {file_name}: {e}')
                continue
            except Exception as e:
                print(f'An unexpected error occurred while reading {file_name}: {e}')
                continue

    if not dataframe_collection:
        print("No data was loaded. Please check your file paths and names.")
        return pd.DataFrame()

    # Concatenate all DataFrames into one DataFrame
    combined_df = pd.concat(dataframe_collection, ignore_index=True)
    combined_df.columns = combined_df.columns.str.strip()  # Strip any leading/trailing whitespace from column names
    return combined_df

def save_to_csv(df, output_path):
    """Save DataFrame to a CSV file."""
    try:
        df.to_csv(output_path, index=False)
        print(f"Data saved successfully to {output_path}")
    except Exception as e:
        print(f"Failed to save the DataFrame: {e}")

# Main execution logic
if __name__ == "__main__":
    source_directory = input('Enter/path/to/data/directory: ')  # Get directory containing the data files from user
    csv_name = input('Enter the filename for the CSV (e.g., combined_data.csv): ')
    output_path = os.path.join(source_directory, csv_name)  # Construct the full path to save the CSV file

    # Load data from the specified directory
    all_data_df = load_data_from_directory(source_directory)

    # Save the data to a CSV file
    if not all_data_df.empty:
        save_to_csv(all_data_df, output_path)
    else:
        print("No data to save.")


Data saved successfully to /Users/nareshchethala/Desktop/University/Sentiment_Analysis/idxfiles/test_combined.csv


# Data Analysis and Visualization of the EDGAR Data


With the Data gathered above, we can analyse with the use of *statistical analysis* and *data visualization*.
We can try to answer the following questions : 
1) Time series counts of filings per year/quarter/month.
2) Is there a seasonality? For instance, are there more filings in a particular month?
3) Top N filers / Top N filing types Summary statistics of filings over quarters/ months: min/max/mean/median/sd of filings.
4) Trend of company filings over the years.

If Your Environment doesn't have the required libraries install, then run the first cell to install the required packages.

In [ ]:
#Importing required Libraries
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta

In [ ]:
#Creating the dataframe from the local repository
file_path = input('Path/to/file.csv:')
df = pd.read_csv(file_path)
df.head()

# Data Modelling

In [ ]:
#Data Cleaning to remove filings with no dates
# Remove rows where 'Date Filed' contains '------------'
df = df[df['Date Filed'] != '------------']

# Printing the data with valid Date column
print(df)

In [ ]:
# Convert 'Date Filed' to datetime if it's not already
df['Date Filed'] = pd.to_datetime(df['Date Filed'])

# Extract month and year from 'Date Filed'
df['Month'] = df['Date Filed'].dt.month
df['Year'] = df['Date Filed'].dt.year
df.head()

In [ ]:
#Creating a dictionary for month names and mapping them to create a new column
month_names = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sept',10:'Oct',11:'Nov',12:'Dec'}
df['Month_Name'] = df['Month'].map(month_names)
df.head()

# Analytics

In [ ]:
#getting summary of each column
df['Form Type'].describe()

In [ ]:
# Convert the 'Month_Name' column to a categorical type with a defined order
df['Month_Name'] = pd.Categorical(df['Month_Name'], categories=month_names.values(), ordered=True)

# Count filings per month
monthly_filings = df.groupby('Month_Name', observed=True).size()

# Sort counts in ascending order(optional, you can remove this, if you want the months to be in sequence)
#monthly_filings_sorted = monthly_filings.sort_values()

# Plotting the results
plt.figure(figsize=(14, 8))
monthly_filings.plot(kind='bar')
plt.title('Number of Filings Per Month', fontsize = 18)
plt.xlabel('Month', fontsize = 16)
plt.ylabel('Number of Filings', fontsize = 16)
plt.xticks(rotation=0, fontsize = 12)
plt.yticks(rotation=0, fontsize = 12)
plt.show()

In [ ]:
# User input for form type
user_form_type = input("Enter the form type you are interested in (e.g., 10-K, 10-Q): ")

# Filter DataFrame for the specific form type
filtered_df = df[df['Form Type'] == user_form_type]

if not filtered_df.empty:
    # Function to find the month with the most filings
    most_frequent_month = filtered_df['Month_Name'].value_counts().idxmax() 
    #idmax is used to get the first one in the list based on index i.e., Month_Name

    print(f"The month with the most filings for {user_form_type} is: {most_frequent_month}")
else:
    print(f"No filings found for form type '{user_form_type}'.")

In [ ]:
# User input for form type
user_form_type = input("Enter the form type you are interested in (e.g., 10-K, 10-Q): ")

# Filter DataFrame for the specific form type
filtered_df = df[df['Form Type'] == user_form_type]

if not filtered_df.empty:
    # Count the number of filings per month
    filings_per_month = filtered_df.groupby('Month_Name', observed = True).size()

    # Plotting the results
    plt.figure(figsize=(12, 6))
    filings_per_month.plot(kind='bar', color='dodgerblue', figsize = (14,8))
    plt.title(f'Number of Filings Per Month for Form - {user_form_type}',fontsize = 18)
    plt.xlabel('Month_Name',fontsize = 16)
    plt.ylabel('Number of Filings',fontsize =16)
    plt.xticks(rotation=0, fontsize = 12)
    plt.yticks(rotation = 45, fontsize = 12)
    plt.show()
else:
    print(f"No filings found for form type '{user_form_type}'.")


In [ ]:
# Requires extra GPU to run
# Calculating different statistics
mean_value = df['Form Type'].mean()
median_value = df['Form Type'].median()
std_dev = df['Form Type'].std()
min_value = df['Form Type'].min()
max_value = df['Form Type'].max()

# Printing the statistics
print(f"Mean (Average): {mean_value}")
print(f"Median: {median_value}")
print(f"Standard Deviation: {std_dev}")
print(f"Minimum Value: {min_value}")
print(f"Maximum Value: {max_value}")

# Getting a summary of statistics
summary = df['Form Type'].describe()
print("\nSummary Statistics:")
print(summary)

In [ ]:
#Optional, run this if you want to get the Form Types into a List
#Creating a list of Form Type
df_form_list = df['Form Type'].unique().tolist()

#Specify the file path where you want to save the list
file_path = input('/Path/to/save/form_list.txt:')

#Writing the list to a text file and saving the file as per the user path
with open(file_path, 'w') as file:
    for item in df_form_list:
        file.write(f"{item}\n")

print(f"List saved to {file_path}")


In [ ]:
#Getting the unique count of the Form Types
dfcount = df['Form Type'].value_counts()
dfcount

In [ ]:
#Plotting the top 20 Form Types with most filings
dfcount[0:20].plot(kind = 'bar', color ='darkblue', figsize =(14,8))
plt.title('Total Number Of Filings Over The Years', fontsize = 18)
plt.xlabel('Form Type', fontsize = 16)
plt.ylabel('Number Of Filings', fontsize = 16)
plt.xticks(rotation=45,  fontsize = 12)
plt.yticks(rotation=0, fontsize = 12)
plt.show()

In [ ]:
#To get the latest Date in the data
latest_date = df['Date Filed'].max()

In [ ]:
# Calculating last 10 years based on the latest_date
ten_years_ago = latest_date - pd.DateOffset(years=10)

In [ ]:
# Filtering data for the last 10 years
df_last_10_years = df[df['Date Filed'] >= ten_years_ago]

In [ ]:
# Calculate frequency of each filing type and get top 10
top_filings = df_last_10_years['Form Type'].value_counts().nlargest(10)
top_filing_types = top_filings.index.tolist()

# Filter the DataFrame to include only top 10 filing types from the last 10 years
df_top10_last10 = df_last_10_years[df_last_10_years['Form Type'].isin(top_filing_types)]

# Group by Form Type and resample by year for trend analysis
trend_analysis = df_top10_last10.groupby('Form Type').resample('YE', on='Date Filed').count()

# Plotting the annual trend analysis as a stacked bar chart
trend_analysis['CIK'].unstack().plot(kind='bar', figsize=(14, 7), stacked=True)
plt.title('Annual Filing Trends by Form Type (Top 10 Over Last 10 Years)',fontsize=18)
plt.ylabel('Number of Filings', fontsize=16)
plt.xlabel('Form Type', fontsize=16)
plt.xticks(rotation=45, fontsize = 12)
plt.yticks(rotation=0, fontsize = 12)
plt.show()

In [ ]:
# Seasonality Analysis for top 10 filings over the last 10 years
df_top10_last10.loc[:, 'Month'] = df_top10_last10['Date Filed'].dt.month
seasonality_analysis = df_top10_last10.groupby(['Month', 'Form Type']).size().unstack()

# Plot the monthly trends for the top 10 filings as a stacked bar chart
seasonality_analysis.plot(kind='bar', figsize=(14, 7), width=0.8, stacked=True)
plt.title('Monthly Filing Trends by Form Type (Top 10 Over Last 10 Years)', fontsize = 18)
plt.ylabel('Number of Filings', fontsize = 16)
plt.xlabel('Month', fontsize = 16)
plt.xticks(ticks=range(0, 12), labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=0, fontsize =12)
plt.yticks(fontsize =12)
plt.show()

In [ ]:
# Count filings per month
yearly_filings = df.groupby('Year').size()

# Sort counts in ascending order(optional, you can remove this, if you want the months to be in sequence)
#monthly_filings_sorted = monthly_filings.sort_values()

# Plotting the results
yearly_filings.plot(kind='bar',color = 'skyblue', figsize =(20,10))
plt.title('Number of Filings Per Year', fontsize = 20)
plt.xlabel('Year', fontsize = 18)
plt.ylabel('Number of Filings', fontsize = 18)
plt.xticks(rotation=90, fontsize = 16)
plt.yticks(rotation=0, fontsize = 16)
plt.show()

## Sentiment Analysis 10_k

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import requests

In [67]:
def download_filings_by_user_input(df, limit=None):
    form_type = input("Enter the SEC form type you want to download (e.g., 10-K, 10-Q, 8-K): ").strip().upper()
    
    filtered_df = df[df['Form Type'].str.upper() == form_type].copy()
    
    if filtered_df.empty:
        print(f"No filings of type '{form_type}' found.")
        return pd.DataFrame()
    
    if limit:
        filtered_df = filtered_df.head(limit)

    filings = []
    base_url = "https://www.sec.gov/Archives/"
    headers = {"User-Agent": "nareshchandra.chethala@gmail.com"}  # Replace with your email for compliance

    for i, row in filtered_df.iterrows():
        filename_clean = row["Filename"].strip().replace(" ", "")
        full_url = base_url + filename_clean
        try:
            response = requests.get(full_url, headers=headers, timeout=10)
            response.raise_for_status()
            filings.append({
                "Company Name": row["Company Name"],
                "CIK": row["CIK"],
                "Form Type": row["Form Type"],
                "Date Filed": row["Date Filed"],
                "Filing URL": full_url,
                "Filing Text": response.text
            })
            print(f"Downloaded: {row['Company Name']} - {row['Form Type']} - {row['Date Filed']}")
        except Exception as e:
            print(f"Failed to download {full_url}: {e}")

    return pd.DataFrame(filings)

In [8]:
df = pd.read_csv("/Users/nareshchethala/Desktop/University/Sentiment_Analysis/idxfiles/combined_data.csv")

In [25]:
filings_df.to_csv("/Users/nareshchethala/Desktop/University/Sentiment_Analysis/idxfiles/html_10k_filings.csv", index=False)

In [ ]:
import requests
from bs4 import BeautifulSoup

def extract_ixbrl_filing_text_from_idx_row(row, user_agent_email):
    try:
        filename = row['Filename'].strip().replace(" ", "")
        path_parts = filename.split("/")

        if len(path_parts) < 4:
            print(f" Invalid path in Filename: {filename}")
            return None, None

        cik = path_parts[2]
        accession_with_dashes = path_parts[3]
        accession_nodashes = accession_with_dashes.replace("-", "")
        index_filename = accession_with_dashes + "-index.htm"

        # Correct index page URL
        index_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{accession_nodashes}/{index_filename}"
        headers = {"User-Agent": user_agent_email}

        response = requests.get(index_url, headers=headers, timeout=10)
        if response.status_code != 200:
            print(f" Failed to load index page: {index_url}")
            return None, None

        # Parse the index page
        soup = BeautifulSoup(response.text, "html.parser")
        doc_table = soup.find("table", class_="tableFile")
        if doc_table is None:
            print(f" Document table not found on: {index_url}")
            return None, None

        # Find the primary .htm (not -index) document
        doc_link_tag = doc_table.find("a", href=lambda href: href and href.endswith(".htm") and not href.endswith("-index.htm"))
        if doc_link_tag is None:
            print(f" No .htm primary document found on: {index_url}")
            return None, None

        primary_doc = doc_link_tag['href'].lstrip("/")
        
        # ✅ Correct iXBRL URL
        ix_url = f"https://www.sec.gov/ix?doc=/{primary_doc}"

        # Download the iXBRL document
        filing_response = requests.get(ix_url, headers=headers, timeout=15)
        if filing_response.status_code == 200:
            print(f" Downloaded: {ix_url}")
            return ix_url, filing_response.text
        else:
            print(f" Failed to download iXBRL filing from: {ix_url}")
            return ix_url, None

    except Exception as e:
        print(f" Exception occurred: {e}")
        return None, None

In [ ]:
# From your .idx DataFrame
sample_row = df[df['Form Type'] == '10-K'].iloc[0]
url, html_text = extract_ixbrl_filing_text_from_idx_row(sample_row, "your_email@example.com")

In [ ]:
# Load your .idx CSV first
df = pd.read_csv("/Users/nareshchethala/Desktop/University/Sentiment_Analysis/idxfiles/combined_data.csv")

# Replace with your actual email
user_email = "nareshchandra.chethala@gmail.com"

# Download first 5 10-K filings
filings_df = download_multiple_10k_filings(df, user_email, limit=5)

# Preview
filings_df.head()

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_ixbrl_filing_text_from_idx_row(row, user_agent_email):
    """
    Extract iXBRL filing text from a single .idx row using SEC iXBRL URL.
    """
    try:
        # Step 1: Clean and parse the filename
        filename = row['Filename'].strip().replace(" ", "")
        path_parts = filename.split("/")

        if len(path_parts) < 4:
            print(f" Invalid path in Filename: {filename}")
            return None, None

        cik = path_parts[2]
        accession_with_dashes = path_parts[3]
        accession_nodashes = accession_with_dashes.replace("-", "")
        index_filename = accession_with_dashes + "-index.htm"

        # Step 2: Construct the index page URL
        index_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{accession_nodashes}/{index_filename}"
        headers = {"User-Agent": user_agent_email}

        response = requests.get(index_url, headers=headers, timeout=10)
        if response.status_code != 200:
            print(f" Failed to load index page: {index_url}")
            return None, None

        # Step 3: Parse HTML to locate the actual .htm document
        soup = BeautifulSoup(response.text, "html.parser")
        doc_table = soup.find("table", class_="tableFile")
        if doc_table is None:
            print(f" Could not find document table at: {index_url}")
            return None, None

        doc_link_tag = doc_table.find("a", href=lambda href: href and href.endswith(".htm") and not href.endswith("-index.htm"))
        if doc_link_tag is None:
            print(f" No .htm filing document found in index page: {index_url}")
            return None, None

        # Remove leading slash if present
        primary_doc = doc_link_tag['href'].lstrip("/")

        # Step 4: Construct the correct iXBRL URL
        ix_url = f"https://www.sec.gov/ix?doc=/{primary_doc}"

        # Step 5: Download the full filing
        filing_response = requests.get(ix_url, headers=headers, timeout=15)
        if filing_response.status_code == 200:
            print(f" Downloaded: {ix_url}")
            return ix_url, filing_response.text  # This is your Filing Text
        else:
            print(f" Failed to download filing from: {ix_url}")
            return ix_url, None

    except Exception as e:
        print(f" Exception occurred: {e}")
        return None, None


def download_multiple_10k_filings(df, user_agent_email, limit=5):
    """
    Download multiple 10-K filings using .idx-based DataFrame.
    """
    tenk_df = df[df['Form Type'].str.upper() == '10-K'].head(limit).reset_index(drop=True)
    results = []

    for idx, row in tenk_df.iterrows():
        url, text = extract_ixbrl_filing_text_from_idx_row(row, user_agent_email)
        if url and text:
            results.append({
                "Company Name": row['Company Name'],
                "CIK": row['CIK'],
                "Date Filed": row['Date Filed'],
                "Filing URL": url,
                "Filing Text": text  # ✅ This is your full HTML text
            })

    return pd.DataFrame(results)

# Working Code

In [ ]:
#This one works
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_filing_html_directly(row, user_agent_email):
    """
    Extracts the actual 10-K filing HTML content from a row in .idx using the real HTML URL.
    """
    try:
        filename = row['Filename'].strip().replace(" ", "")
        path_parts = filename.split("/")

        if len(path_parts) < 4:
            print(f"Invalid path in Filename: {filename}")
            return None, None

        cik = path_parts[2]
        accession_with_dashes = path_parts[3]
        accession_nodashes = accession_with_dashes.replace("-", "")
        index_filename = accession_with_dashes + "-index.htm"

        index_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{accession_nodashes}/{index_filename}"
        headers = {"User-Agent": user_agent_email}

        response = requests.get(index_url, headers=headers, timeout=10)
        if response.status_code != 200:
            print(f"Failed to load index page: {index_url}")
            return None, None

        soup = BeautifulSoup(response.text, "html.parser")
        doc_table = soup.find("table", class_="tableFile")
        if doc_table is None:
            print(f"Could not find document table at: {index_url}")
            return None, None

        doc_link_tag = doc_table.find("a", href=lambda href: href and href.endswith(".htm") and not href.endswith("-index.htm"))
        if doc_link_tag is None:
            print(f"No .htm filing document found in index page: {index_url}")
            return None, None

        primary_doc = doc_link_tag['href'].lstrip("/")  # remove leading slash
        filing_url = f"https://www.sec.gov/{primary_doc}"  # FIXED — no double Archives

        filing_response = requests.get(filing_url, headers=headers, timeout=15)
        if filing_response.status_code == 200:
            print(f"Downloaded: {filing_url}")
            return filing_url, filing_response.text
        else:
            print(f"Failed to download filing from: {filing_url}")
            return filing_url, None

    except Exception as e:
        print(f"Exception occurred: {e}")
        return None, None

In [5]:
def download_multiple_10k_filings(df, user_agent_email):
    """
    Show how many 10-Ks are available, let the user choose how many to download,
    and return a DataFrame with filing metadata and text.
    """
    # Step 1: Filter for 10-Ks
    tenk_df = df[df['Form Type'].str.upper() == '10-K'].reset_index(drop=True)
    total = len(tenk_df)

    if total == 0:
        print("No 10-K filings found in the dataset.")
        return pd.DataFrame()

    print(f"Found {total} 10-K filings in the dataset.")
    
    # Step 2: Ask user for how many to download
    while True:
        try:
            limit = int(input(f"Enter the number of 10-K filings to download (1 to {total}): "))
            if 1 <= limit <= total:
                break
            else:
                print(f"Please enter a number between 1 and {total}.")
        except ValueError:
            print("Please enter a valid integer.")

    # Step 3: Download filings
    results = []
    for idx, row in tenk_df.head(limit).iterrows():
        url, html_text = extract_filing_html_directly(row, user_agent_email)
        if html_text:
            results.append({
                "Company Name": row['Company Name'],
                "CIK": row['CIK'],
                "Date Filed": row['Date Filed'],
                "Filing URL": url,
                "Filing Text": html_text
            })

    return pd.DataFrame(results)

In [9]:
df= pd.read_csv("/Users/nareshchethala/Desktop/University/Sentiment_Analysis/idxfiles/test_combined.csv")

In [10]:
df.head()

,Company Name,Form Type,CIK,Date Filed,Filename
0,--------------------------------------------------------------,------------,------------,------------,-------------------------------------------
1,1 800 FLOWERS COM INC,10-K,1084869,2020-09,-11 edgar/data/1084869/0001437749-20-019622.txt
2,1 800 FLOWERS COM INC,3,1084869,2020-07,-02 edgar/data/1084869/0001437749-20-014500.txt
3,1 800 FLOWERS COM INC,4,1084869,2020-09,-02 edgar/data/1084869/0001437749-20-019189.txt
4,1 800 FLOWERS COM INC,4,1084869,2020-09,-02 edgar/data/1084869/0001437749-20-019193.txt


In [11]:
# Run this after loading your .idx dataframe
filings_df = download_multiple_10k_filings(df, "nareshchandra.chethala@gmail.com")

# Preview
print(filings_df[['Company Name', 'Filing URL']].head())

Found 35986 10-K filings in the dataset.
Downloaded: https://www.sec.gov/Archives/edgar/data/1084869/000143774920019622/flws20200628b_10k.htm
Downloaded: https://www.sec.gov/Archives/edgar/data/1667615/000107878220000695/f10k053120_10k.htm
Downloaded: https://www.sec.gov/Archives/edgar/data/1753648/000149315220017860/form10-k.htm
Downloaded: https://www.sec.gov/Archives/edgar/data/1591588/000156459020043316/amrk-10k_20200630.htm
Downloaded: https://www.sec.gov/ix?doc=/Archives/edgar/data/1750/000110465920085310/air-20200531x10k.htm
Downloaded: https://www.sec.gov/Archives/edgar/data/1300938/000118518520001098/abcoenergy20191231_10k.htm
Downloaded: https://www.sec.gov/ix?doc=/Archives/edgar/data/1138723/000156459020041201/aray-10k_20200630.htm
Downloaded: https://www.sec.gov/Archives/edgar/data/849401/000143774920015110/admt20200331_10k.htm
Downloaded: https://www.sec.gov/Archives/edgar/data/828530/000155335020000885/adfk_10k.htm
Downloaded: https://www.sec.gov/Archives/edgar/data/35299

In [ ]:
sample_row = df[df['Form Type'] == '10-K'].iloc[0]
url, html_text = extract_filing_html_directly(sample_row, "nareshchandra.chethala@gmail.com")

if html_text:
    print(" Filing text preview:\n")
    print(html_text[:1000])
else:
    print("No filing text returned.")

Downloaded: https://www.sec.gov/Archives/edgar/data/1084869/000143774920019622/flws20200628b_10k.htm
✅ Filing text preview:

<DOCUMENT>
<TYPE>10-K
<SEQUENCE>1
<FILENAME>flws20200628b_10k.htm
<DESCRIPTION>FORM 10-K
<TEXT>
<html><head>
	<title>flws20200628b_10k.htm</title>


	<!-- Generated by ThunderDome Portal - 9/11/2020 5:19:29 PM --><meta charset="utf-8"><meta name="format-detection" content="telephone=no">
</head>
<body style="font-family: &quot;Times New Roman&quot;, Times, serif; font-size: 10pt; padding: 0in 0.1in; cursor: auto;">
<div contenteditable="false" docsect="2" id="143103" sectdesc="Cover Page">&nbsp;</div>

<div data-td-statement-type="cover" data-td-v2v-id="1599662362529">
<p style="font-family:'Times New Roman', Times, serif;font-size:10pt;margin:0pt 7.2pt;text-align:justify;">&nbsp;</p>

<p style="font-family: &quot;Times New Roman&quot;, Times, serif; font-size: 10pt; margin: 0pt; text-align: justify;"><a href="#toc" style="font-family: &quot;Times New Roman&quot;

In [57]:
print(html_text)

<DOCUMENT>
<TYPE>10-K
<SEQUENCE>1
<FILENAME>flws20200628b_10k.htm
<DESCRIPTION>FORM 10-K
<TEXT>
<html><head>
	<title>flws20200628b_10k.htm</title>


	<!-- Generated by ThunderDome Portal - 9/11/2020 5:19:29 PM --><meta charset="utf-8"><meta name="format-detection" content="telephone=no">
</head>
<body style="font-family: &quot;Times New Roman&quot;, Times, serif; font-size: 10pt; padding: 0in 0.1in; cursor: auto;">
<div contenteditable="false" docsect="2" id="143103" sectdesc="Cover Page">&nbsp;</div>

<div data-td-statement-type="cover" data-td-v2v-id="1599662362529">
<p style="font-family:'Times New Roman', Times, serif;font-size:10pt;margin:0pt 7.2pt;text-align:justify;">&nbsp;</p>

<p style="font-family: &quot;Times New Roman&quot;, Times, serif; font-size: 10pt; margin: 0pt; text-align: justify;"><a href="#toc" style="font-family: &quot;Times New Roman&quot;, Times, serif; font-size: 10pt">Table of Contents</a></p>

<p style="font-family:'Times New Roman', Times, serif;font-size:1

In [14]:
from bs4 import BeautifulSoup

def clean_filing_html(filing_html):
    """
    Cleans the full HTML of a 10-K filing to extract readable plain text.
    Removes scripts, styles, and unnecessary whitespace.
    """
    try:
        soup = BeautifulSoup(filing_html, "html.parser")

        # Remove unwanted tags
        for tag in soup(["script", "style", "header", "footer", "nav", "noscript"]):
            tag.decompose()

        # Extract text from the body if present
        body = soup.find("body")
        raw_text = body.get_text(separator="\n") if body else soup.get_text(separator="\n")

        # Normalize whitespace
        lines = [line.strip() for line in raw_text.splitlines()]
        clean_text = "\n".join(line for line in lines if line)

        return clean_text

    except Exception as e:
        print(f" Error cleaning HTML: {e}")
        return ""

In [ ]:
clean_text = clean_filing_html(html_text)
print(" Cleaned text preview:\n")
print(clean_text)

✅ Cleaned text preview:

Table of Contents
UNITED STATES SECURITIES AND EXCHANGE COMMISSION
WASHINGTON, D.C. 20549
FORM 10-K
☒     ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the fiscal year ended
June 28, 2020
or
☐     TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
Commission File No.
0-26841
1-800-FLOWERS.COM, Inc.
(Exact name of registrant as specified in its charter)
DELAWARE
(State or other jurisdiction of incorporation or organization)
11-3117311
(I.R.S. Employer Identification No.)
One Old Country Road, Carle Place, New York, 11514
(Address of principal executive offices) (Zip code)
(516) 237-6000
(Registrant’s telephone number, including area code)
Securities registered pursuant to Section 12(b) of the Act:
Title of each class
Trading symbol(s)
Name of each exchange on which registered
Class A common stock
FLWS
The Nasdaq Stock Market
Securities registered pursuant to Section 12(g) of the Act

In [62]:
print(len(clean_text))

286532


In [ ]:
def download_multiple_10k_filings(df, user_agent_email):
    """
    Show how many 10-Ks are available, let the user choose how many to download,
    and return a DataFrame with filing metadata and text.
    """
    tenk_df = df[df['Form Type'].str.upper() == '10-K'].reset_index(drop=True)
    total = len(tenk_df)

    if total == 0:
        print(" No 10-K filings found in the dataset.")
        return pd.DataFrame()

    print(f"Found {total} 10-K filings in the dataset.")
    
    while True:
        try:
            limit = int(input(f"Enter the number of 10-K filings to download (1 to {total}): "))
            if 1 <= limit <= total:
                break
            else:
                print(f"Please enter a number between 1 and {total}.")
        except ValueError:
            print("Please enter a valid integer.")

    results = []
    for idx, row in tenk_df.head(limit).iterrows():
        url, html_text = extract_filing_html_directly(row, user_agent_email)
        if html_text:
            cleaned_text = clean_filing_html(html_text)
            results.append({
                "Company Name": row['Company Name'],
                "CIK": row['CIK'],
                "Date Filed": row['Date Filed'],
                "Filing URL": url,
                "Filing Text": html_text,
                "Cleaned Text": cleaned_text
            })

    return pd.DataFrame(results)

In [ ]:
def download_multiple_10k_filings(df, user_agent_email):
    """
    Show how many 10-Ks are available, let the user choose how many to download,
    and return a DataFrame with filing metadata and text.
    """
    tenk_df = df[df['Form Type'].str.upper() == '10-K'].reset_index(drop=True)
    total = len(tenk_df)

    if total == 0:
        print("No 10-K filings found in the dataset.")
        return pd.DataFrame()

    print(f"🔍 Found {total} 10-K filings in the dataset.")
    
    while True:
        try:
            limit = int(input(f"Enter the number of 10-K filings to download (1 to {total}): "))
            if 1 <= limit <= total:
                break
            else:
                print(f"⚠️ Please enter a number between 1 and {total}.")
        except ValueError:
            print("⚠️ Please enter a valid integer.")

    results = []
    for idx, row in tenk_df.head(limit).iterrows():
        url, html_text = extract_filing_html_directly(row, user_agent_email)
        if html_text:
            cleaned_text = clean_filing_html(html_text)
            results.append({
                "Company Name": row['Company Name'],
                "CIK": row['CIK'],
                "Date Filed": row['Date Filed'],
                "Filing URL": url,
                "Filing Text": html_text,
                "Cleaned Text": cleaned_text
            })

    return pd.DataFrame(results)

In [19]:
# Run the full process
filings_df = download_multiple_10k_filings(df, "nareshchandra.chethala@gmail.com")

# Preview results
print(filings_df[["Company Name", "Filing URL"]].head())
print("\n✅ Sample Cleaned Filing Text:\n")
print(filings_df["Cleaned Text"][0][:2000])

🔍 Found 35986 10-K filings in the dataset.
Downloaded: https://www.sec.gov/Archives/edgar/data/1084869/000143774920019622/flws20200628b_10k.htm
Downloaded: https://www.sec.gov/Archives/edgar/data/1667615/000107878220000695/f10k053120_10k.htm
Downloaded: https://www.sec.gov/Archives/edgar/data/1753648/000149315220017860/form10-k.htm
Downloaded: https://www.sec.gov/Archives/edgar/data/1591588/000156459020043316/amrk-10k_20200630.htm
Downloaded: https://www.sec.gov/ix?doc=/Archives/edgar/data/1750/000110465920085310/air-20200531x10k.htm
Downloaded: https://www.sec.gov/Archives/edgar/data/1300938/000118518520001098/abcoenergy20191231_10k.htm
Downloaded: https://www.sec.gov/ix?doc=/Archives/edgar/data/1138723/000156459020041201/aray-10k_20200630.htm
Downloaded: https://www.sec.gov/Archives/edgar/data/849401/000143774920015110/admt20200331_10k.htm
Downloaded: https://www.sec.gov/Archives/edgar/data/828530/000155335020000885/adfk_10k.htm
Downloaded: https://www.sec.gov/Archives/edgar/data/352

In [23]:
filings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Company Name  20 non-null     object
 1   CIK           20 non-null     object
 2   Date Filed    20 non-null     object
 3   Filing URL    20 non-null     object
 4   Filing Text   20 non-null     object
 5   Cleaned Text  20 non-null     object
dtypes: object(6)
memory usage: 1.1+ KB


In [25]:
filings_df["Cleaned Text"].head(1)
#print(filings_df["Cleaned Text"][0][:2000])

0    Table of Contents\nUNITED STATES SECURITIES AND EXCHANGE COMMISSION\nWASHINGTON, D.C. 20549\nFORM 10-K\n☒     ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended\nJune 28, 2020\nor\n☐     TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nCommission File No.\n0-26841\n1-800-FLOWERS.COM, Inc.\n(Exact name of registrant as specified in its charter)\nDELAWARE\n(State or other jurisdiction of incorporation or organization)\n11-3117311\n(I.R.S. Employer Identification No.)\nOne Old Country Road, Carle Place, New York, 11514\n(Address of principal executive offices) (Zip code)\n(516) 237-6000\n(Registrant’s telephone number, including area code)\nSecurities registered pursuant to Section 12(b) of the Act:\nTitle of each class\nTrading symbol(s)\nName of each exchange on which registered\nClass A common stock\nFLWS\nThe Nasdaq Stock Market\nSecurities registered pursuant to Section 12(g) of